In [1]:
!pip install javascript

  Using cached javascript-1%211.2.6-py3-none-any.whl.metadata (14 kB)
Using cached javascript-1%211.2.6-py3-none-any.whl (34 kB)


In [ ]:
from javascript import require, On, Once, AsyncTask, once, off
mineflayer = require('mineflayer')
random_number = id([]) % 1000 # Give us a random number upto 1000
BOT_USERNAME = f'colab_{random_number}'

bot = mineflayer.createBot({ 'host': '127.0.0.1', 'port': 25565, 'username': BOT_USERNAME, 'hideErrors': False })

# The spawn event
once(bot, 'login')
bot.chat('I spawned')
bot.entity.position
@On(bot, 'playerJoin')
def end(this, player):
  bot.chat('Someone joined!')

@On(bot, 'chat')
def onChat(this, user, message, *rest):
  print(f'{user} said "{message}"')

  # If the message contains stop, remove the event listener and stop logging.
  if 'stop' in message:
    off(bot, 'chat', onChat)

@On(bot, 'chat')
def breakListener(this, sender, message, *args):
  if sender and (sender != BOT_USERNAME):
    if 'break' in message:
      pos = bot.entity.position.offset(0, -1, 0)
      blockUnder = bot.blockAt(pos)
      if bot.canDigBlock(blockUnder):
        bot.chat(f"I'm breaking the '{blockUnder.name}' block underneath")
        # The start=True parameter means to immediately invoke the function underneath
        # If left blank, you can start it with the `start()` function later on.
        try:
          @AsyncTask(start=True)
          def break_block(task):
            bot.dig(blockUnder)
          bot.chat('I started digging!')
        except Exception as e:
          bot.chat(f"I had an error {e}")
      else:
        bot.chat(f"I can't break the '{blockUnder.name}' block underneath")
    if 'stop' in message:
      off(bot, 'chat', breakListener)
      
pathfinder = require('mineflayer-pathfinder')
bot.loadPlugin(pathfinder.pathfinder)
# Create a new minecraft-data instance with the bot's version
mcData = require('minecraft-data')(bot.version)
# Create a new movements class
movements = pathfinder.Movements(bot, mcData)
# How far to be from the goal
RANGE_GOAL = 1

bot.removeAllListeners('chat')
@On(bot, 'chat')
def handleMsg(this, sender, message, *args):
  if sender and (sender != BOT_USERNAME):
    bot.chat('Hi, you said ' + message)
    if 'come' in message:
      player = bot.players[sender]
      target = player.entity
      if not target:
        bot.chat("I don't see you !")
        return
      pos = target.position
      bot.pathfinder.setMovements(movements)
      bot.pathfinder.setGoal(pathfinder.goals.GoalNear(pos.x, pos.y, pos.z, RANGE_GOAL))
    if 'stop' in message:
      off(bot, 'chat', handleMsg)

import matplotlib.pyplot as plt
figure = plt.figure()
axes = figure.add_axes([0,0,1,1])
Vec3 = require('vec3').Vec3

columns = bot.world.getColumns()
block_freqs = {}
for c in range(0, 3): # iterate through some of the loaded chunk columns
  cc = columns[c].column
  for y in range(1, 40):
    for x in range(1, 16):
      for z in range(1, 16):
        block = cc.getBlock(Vec3(x, y, z))
        if block.name in block_freqs:
          block_freqs[block.name] += 1
        else:
          block_freqs[block.name] = 1

print(block_freqs)
axes.bar(block_freqs.keys(), block_freqs.values())
plt.xticks(rotation=45)
plt.show()

bot.quit()